In [ ]:
!pip install transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Startfrom7500.csv to Startfrom7500.csv


In [ ]:
import pandas as pd
import torch
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import os
from google.colab import drive
drive.mount('/content/drive')

print(os.path.exists("/content/drive/My Drive/Models/model2"))


# Path to your model directory (replace with your actual path)
model_directory = '/content/drive/My Drive/Models/model2'  # Update with your actual path

# Load the trained model and tokenizer
model = RobertaForSequenceClassification.from_pretrained(model_directory)
tokenizer = RobertaTokenizer.from_pretrained(model_directory)

# Ensure your model is in evaluation mode
model.eval()

# Set the device to 'cuda' if available, otherwise use 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the CSV file containing the comments (replace 'your_file.csv' with your actual file name)
file_path = 'Startfrom7500.csv'  # Replace with your actual uploaded file name
df = pd.read_csv(file_path)

# Check the first few rows to ensure the 'comment' column exists
print(df.head())

# Assuming your CSV has a column named 'comment', extract it
comments = df['comment'].tolist()

# Define a smaller batch size (e.g., 8 or 16)
batch_size = 8  # You can adjust this depending on available memory

# Initialize a list to store the predictions
predicted_labels = []

# Process the comments in batches to avoid memory overload
for i in range(0, len(comments), batch_size):
    # Get the current batch of comments
    batch_comments = comments[i:i + batch_size]

    # Tokenize the current batch
    inputs = tokenizer(batch_comments, padding=True, truncation=True, return_tensors="pt", max_length=512)

    # Move the inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Make predictions with the trained model
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

    # Map predictions to human-readable labels (adjust based on your model's label mapping)
    labels = ['positive', 'negative', 'neutral']  # Adjust according to your model's label mapping
    predicted_labels.extend([labels[pred.item()] for pred in predictions])

# Add the predicted labels to the DataFrame
df['predicted_sentiment'] = predicted_labels

# Display the resulting DataFrame
print(df[['comment', 'predicted_sentiment']].head())  # Display the first few rows

Mounted at /content/drive
True
     ID                                            comment sentiment
0  7500                         အဆုံးသတ်ကြီးကမမိုက်ဘူးကွာ💔       NaN
1  7501         မှာလို့ရှာလိုက်ထူးဆန်းတာတွေတွေ့လိမ့်မယ်😮😮😮       NaN
2  7503                             ယောက်ကားတွေမကောင်းဘူး😅       NaN
3  7504  ကိုယ်တွေဦးလေးလဲရှမ်းပြည်ကစော်လေးနဲ့ကြိုက်တာဘဲ...       NaN
4  7506                    တောင်းကြိးမာ သွာလုပ်တာ ဒီဘဲကွာ😒       NaN
                                             comment predicted_sentiment
0                         အဆုံးသတ်ကြီးကမမိုက်ဘူးကွာ💔            negative
1         မှာလို့ရှာလိုက်ထူးဆန်းတာတွေတွေ့လိမ့်မယ်😮😮😮             neutral
2                             ယောက်ကားတွေမကောင်းဘူး😅             neutral
3  ကိုယ်တွေဦးလေးလဲရှမ်းပြည်ကစော်လေးနဲ့ကြိုက်တာဘဲ...            positive
4                    တောင်းကြိးမာ သွာလုပ်တာ ဒီဘဲကွာ😒             neutral


In [ ]:
# Save the DataFrame with predictions to a new CSV file
output_file_path = '/content/Predicted1000.csv'
df.to_csv(output_file_path, index=False)

# Provide a link to download the CSV file
from google.colab import files
files.download(output_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>